In [ ]:
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template
import torch
import pandas as pd
from transformers import AutoTokenizer

In [ ]:
MODEL_NAME = "nraesalmi/phi3.5_set_eval_adapters"
BASE_MODEL = "unsloth/Phi-3.5-mini-instruct-bnb-4bit"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

csv_file = "C:\\Users\\nikke\\GitHub\\ai-pentest-report-finetuning-pipeline\\data\\manual_model_eval_dataset_0.3k.csv"

In [ ]:
# Load your CSV
data = pd.read_csv(csv_file)

# Extract instruction and expected output columns
instructions = data["input"].astype(str)
expected_outputs = data["output"].astype(str)

In [ ]:
# Load model + tokenizer via Unsloth
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = BASE_MODEL,
    max_seq_length = 8192,     # adjust if needed 8192
    dtype = None,              # auto
    load_in_4bit = True,      # set True if VRAM constrained
)

model.load_adapter(MODEL_NAME)


# Enable inference optimizations
FastLanguageModel.for_inference(model)

print(model.peft_config)

In [ ]:
import re

def split_sections(text):
    return [s.strip() for s in re.split(r"\n\s*\n", text) if s.strip()]

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

sentencemodel = SentenceTransformer("all-MiniLM-L6-v2")

def embed_sections(sections):
    return sentencemodel.encode(sections, normalize_embeddings=True)

In [ ]:
def section_similarity(expected_sections, output_sections):
    exp_emb = embed_sections(expected_sections)
    out_emb = embed_sections(output_sections)

    sims = []
    for e in exp_emb:
        sims.append(np.max(out_emb @ e))
    return np.mean(sims)

In [ ]:
from transformers import TextStreamer

# Set up tokenizer template
tokenizer = get_chat_template(
    tokenizer,
    chat_template="phi-3"
)

# Enable faster inference
FastLanguageModel.for_inference(model)

i=0
matches = 0
outputs = []

# Limit to first 10 for testing
# instructions = instructions.head(10)

for instr in instructions:

    expected_output = expected_outputs.iloc[i]
    print(f"\n\n=== Example {i+1} ===")
    print("----------------- Expected Output ----------------\n", expected_output)
    print ("\n------------------- Model Output ------------------\n")

    # Example JSON summarization message with rules
    messages = [
        {"role": "system", "content": """
You are an AI penetration test summarizing assistant.

You MUST output the following exact structure:

## Issue Summary:
<one sentence>

### Remediation Recommendation:
<one sentence>

Rules:
- Exactly two sentences total.
- Each sentence must be on its own line under the correct header.
- Do not combine sentences.
- Do not add text before or after the headers.
- Do not explain. 
"""},
        {"role": "user", "content": f"""Here is the penetration test summary:\n{instr}"""}
    ]

    # Tokenize with attention mask
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    )

    # Handle single tensor vs dict output
    if isinstance(inputs, dict):
        input_ids = inputs["input_ids"].to("cuda")
        attention_mask = inputs["attention_mask"].to("cuda")
    else:
        input_ids = inputs.to("cuda")
        attention_mask = None

    eos_token_id = tokenizer.convert_tokens_to_ids("<|end|>")

    # Generate output
    model_output = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=100,
        use_cache=True,
        do_sample=False,
        temperature=0,
        top_p=0.9,
        repetition_penalty=1.2,
        eos_token_id=eos_token_id,
    )

    # Compare model output to expected output
    generated_ids = model_output[0][input_ids.shape[-1]:]

    decoded_output = tokenizer.decode(
        generated_ids,
        skip_special_tokens=True
    )

    for end_token in ["<|endoftext|>", "<end>"]:
        decoded_output = decoded_output.replace(end_token, "")

    # Strip extra whitespace
    truncate_output = decoded_output.strip()

    print(truncate_output)

    outputs.append(truncate_output)
    i += 1

    # Calculate similarity score
    score = section_similarity(
        split_sections(expected_output),
        split_sections(truncate_output)
    )

    accuracy = score * 100


    print(f"Accuracy: {accuracy:.2f}%")

    if accuracy >= 70:
        print("Result: ✅ Match")
        matches += 1
    else:
        print("Result: ❌ Mismatch")


print(f"\n\n=== Summary ===\nTotal Matches: {matches}/{len(instructions)}")
pd.DataFrame({"model_output": outputs}).to_csv("../data/ai_outputs.csv", index=False)